In [11]:
import rebound
import numpy as np

In [12]:
import os

os.environ["https_proxy"] = "http://localhost:49490"
os.environ["http_proxy"] = "http://localhost:49490"
os.environ["all_proxy"] = "socks5://localhost:49490"
os.environ["HTTPS_PROXY"] = "http://localhost:49490"
os.environ["HTTP_PROXY"] = "http://localhost:49490"
os.environ["ALL_PROXY"] = "socks5://localhost:49490"



In [13]:

import rebound
import time

%load_ext autoreload
%autoreload 2

from utils import (
    retry_on_exception,
    sim_logger,
    fetch_solar_system_bodies
)

sim_logger.setLevel("DEBUG")


# Details about the units and gravitational constant: https://rebound.readthedocs.io/en/latest/ipython_examples/Units/
# Yr = 365.25 days (Julian)
# AU = 1.495978707e+11 m\
# Msun = 1.98855e+30 kg
# G = 4 * pi**2 (AU**3 / Msun / Yr**2) = 39.47841760435743

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# new_celestials = fetch_solar_system_bodies()

# import json

# # Convert the dictionary to a JSON string with maximum precision
# json_str = json.dumps(new_celestials, ensure_ascii=False, indent=None, separators=(',', ':'), sort_keys=True, default=lambda x: format(x, ".15g"))

# # Save to a JSONL file
# with open("solar_system_data.jsonl", "w") as file:
#     file.write(json_str + "\n")  # Add a newline to make it JSONL format

In [18]:
import json

celestial_bodies = json.load(open("solar_system_data.jsonl", "r"))

In [20]:

next(iter(celestial_bodies.values()))
# add up all the masses
total_mass = sum([body["m"] for body in celestial_bodies.values()])
total_mass

1.0013418309074729

In [ ]:
def nbody_simulation_3d(celestial_bodies,
               tmax=1e4,
               dt=1e-2, 
               integrator="whfast",
               collision="direct",
               collision_resolve="merge",
               filename="rebound.bin"):
    
    sim = rebound.Simulation()
    sim.units = ("yr", "AU", "Msun")
    sim.integrator = integrator
    sim.dt = dt
    sim.add(celestial_bodies.values())
    
    # move to the center of mass frame
    sim.move_to_com()
    
    sim.automateSimulationArchive(filename, interval=1e3, deletefile=True)
    
    sim.exit_min_distance = 0.01
    
    # collision detection
    sim.collision = "direct"
    sim.collision_resolve = "merge"
    sim.collision_resolve_keep_sorted = 1
    sim.collision_resolve_keep_sorted = 1
    sim.collision_resolve

In [21]:
import numpy as np
import scipy
solve_ivp = scipy.integrate.solve_ivp
import os, sys

In [ ]:
# ##### ENERGY #####
# def potential_energy(state):
#     '''U=\sum_i,j>i G m_i m_j / r_ij'''
#     tot_energy = np.zeros((1,1,state.shape[2]))
#     for i in range(state.shape[0]):
#         for j in range(i+1,state.shape[0]):
#             r_ij = ((state[i:i+1,1:3] - state[j:j+1,1:3])**2).sum(1, keepdims=True)**.5
#             m_i = state[i:i+1,0:1]
#             m_j = state[j:j+1,0:1]
#             tot_energy += m_i * m_j / r_ij
#     U = -tot_energy.sum(0).squeeze()
#     return U


# def kinetic_energy(state):
#     '''T=\sum_i .5*m*v^2'''
#     energies = .5 * state[:,0:1] * (state[:,3:5]**2).sum(1, keepdims=True)
#     T = energies.sum(0).squeeze()
#     return T


# def total_energy(state):
#     return potential_energy(state) + kinetic_energy(state)

##### DYNAMICS #####
def get_accelerations(state, epsilon=0):
    # shape of state is [bodies x properties]
    net_accs = [] # [nbodies x 2]
    for i in range(state.shape[0]): # number of bodies
        other_bodies = np.concatenate([state[:i, :], state[i+1:, :]], axis=0)
        displacements = other_bodies[:, 1:3] - state[i, 1:3] # indexes 1:3 -> pxs, pys
        distances = (displacements**2).sum(1, keepdims=True)**0.5
        masses = other_bodies[:, 0:1] # index 0 -> mass
        pointwise_accs = masses * displacements / (distances**3 + epsilon) # G=1
        net_acc = pointwise_accs.sum(0, keepdims=True)
        net_accs.append(net_acc)
    net_accs = np.concatenate(net_accs, axis=0)
    return net_accs
  
def update(t, state):
    state = state.reshape(-1,5) # [bodies, properties]
    deriv = np.zeros_like(state)
    deriv[:,1:3] = state[:,3:5] # dx, dy = vx, vy
    deriv[:,3:5] = get_accelerations(state)
    return deriv.reshape(-1)

In [ ]:
##### INTEGRATION SETTINGS #####
def get_orbit(state, update_fn=update, t_points=100, t_span=[0,2], nbodies=3, **kwargs):
    if not 'rtol' in kwargs.keys():
        kwargs['rtol'] = 1e-9

    orbit_settings = locals()

    nbodies = state.shape[0]
    t_eval = np.linspace(t_span[0], t_span[1], t_points)
    orbit_settings['t_eval'] = t_eval

    path = solve_ivp(fun=update_fn, t_span=t_span, y0=state.flatten(),
                     t_eval=t_eval, **kwargs)
    orbit = path['y'].reshape(nbodies, 5, t_points)
    return orbit, orbit_settings

In [ ]:
##### INITIALIZE THE TWO BODIES #####
def rotate2d(p, theta):
  c, s = np.cos(theta), np.sin(theta)
  R = np.array([[c, -s],[s, c]])
  return (R @ p.reshape(2,1)).squeeze()

def random_config(nu=2e-1, min_radius=0.9, max_radius=1.2):
  '''This is not principled at all yet'''
  state = np.zeros((3,5))
  state[:,0] = 1
  p1 = 2 * np.random.rand(2) - 1 # random position in [-1,1] x [-1,1]
  r = np.random.rand() * (max_radius-min_radius) + min_radius
  
  p1 *= r/np.sqrt( np.sum((p1**2)) )
  p2 = rotate2d(p1, theta=2*np.pi/3)
  p3 = rotate2d(p2, theta=2*np.pi/3)

  # # velocity that yields a circular orbit
  v1 = rotate2d(p1, theta=np.pi/2)
  v1 = v1 / r**1.5
  v1 = v1 * np.sqrt(np.sin(np.pi/3)/(2*np.cos(np.pi/6)**2)) # scale factor to get circular trajectories
  v2 = rotate2d(v1, theta=2*np.pi/3)
  v3 = rotate2d(v2, theta=2*np.pi/3)
  
  # make the circular orbits slightly chaotic
  v1 *= 1 + nu*(2*np.random.rand(2) - 1)
  v2 *= 1 + nu*(2*np.random.rand(2) - 1)
  v3 *= 1 + nu*(2*np.random.rand(2) - 1)

  state[0,1:3], state[0,3:5] = p1, v1
  state[1,1:3], state[1,3:5] = p2, v2
  state[2,1:3], state[2,3:5] = p3, v3
  return state

In [ ]:
##### INTEGRATE AN ORBIT OR TWO #####
def sample_orbits(timesteps=20, trials=5000, nbodies=3, orbit_noise=2e-1,
                  min_radius=0.9, max_radius=1.2, t_span=[0, 5], verbose=False, **kwargs):
    
    orbit_settings = locals()
    if verbose:
        print("Making a dataset of near-circular 3-body orbits:")
    
    x, dx, e = [], [], []
    N = timesteps * trials
    while len(x) < N:

        state = random_config(nu=orbit_noise, min_radius=min_radius, max_radius=max_radius)
        orbit, settings = get_orbit(state, t_points=timesteps, t_span=t_span, nbodies=nbodies, **kwargs)
        batch = orbit.transpose(2,0,1).reshape(-1,nbodies*5)
        
        for state in batch:
            dstate = update(None, state)
            
            # reshape from [nbodies, state] where state=[m, qx, qy, px, py]
            # to [canonical_coords] = [qx1, qx2, qy1, qy2, px1,px2,....]
            
            coords = state.reshape(nbodies,5).T[1:].flatten()
            x.append(coords)

    data = {'coords': np.stack(x)[:N]}
    
    return data, orbit_settings

In [ ]:
# import numpy as np

# def initialize_solar_system(n=9, sun_mass=1e6, min_dist=0.9, max_dist=1.2, nu=2e-1):
#     '''Initialize a mock solar system with n bodies'''
    
#     # Initial state [m, vx, vy, vz, x, y, z]
#     state = np.zeros((n, 7))
    
#     # Set the Sun at the center with a large mass and zero velocity
#     state[0, 0] = sun_mass  # Sun's mass
#     state[0, 4:7] = 0  # Sun's position

#     for i in range(1, n):
#         # Set mass (considerably less than the Sun)
#         state[i, 0] = 1  # Example mass; can be randomized
        
#         # Random distance from the Sun
#         r = np.random.rand() * (max_dist - min_dist) + min_dist
        
#         # Random position in 3D space but at distance r from Sun
#         rand_dir = np.random.randn(3)  # Random direction
#         rand_dir /= np.linalg.norm(rand_dir)  # Normalize to unit vector
#         state[i, 4:7] = r * rand_dir
        
#         # Velocity for roughly circular orbit, perturbed slightly for randomness
#         v = np.cross([0, 0, 1], state[i, 4:7])
#         v /= np.linalg.norm(v)
#         v *= np.sqrt(sun_mass / r**3)
        
#         # Make orbits slightly chaotic
#         v *= 1 + nu*(2*np.random.rand(3) - 1)
#         state[i, 1:4] = v

#     return state

In [ ]:
data = sample_orbits(timesteps=200, trials=5, nbodies=3, orbit_noise=2e-1, min_radius=0.9, max_radius=1.2, t_span=[0, 5], verbose=True)

# print(data.keys())
# print(data['coords'].shape)
# print(data['dcoords'].shape)
# print(data['energy'].shape)
# print(data['meta'])